In [1]:
pip install cohere requests tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for cohere: filename=cohere-2.2.5-cp37-cp37m-linux_x86_64.whl size=10453 sha256=2dc20f86394c857044f582c70b13eaeda5d610ba24cd8b9ef18eb842ad486351
  Stored in directory: /root/.cache/pip/wheels/eb/19/2d/89553daf06bc949f1aeb4cfae6e1e306d7763b22ffe91e45fa
Successfully built cohere


We'll then import these packages and declare the function that retrieves post titles from reddit.

In [2]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)

df  = pd.read_excel('./data/Example_data.xlsx')
df['Description'].head(10)

FileNotFoundError: [Errno 2] No such file or directory: './data/Example_data.xlsx'

You'll need your API key for this next cell. [Sign up to Cohere](https://os.cohere.ai/) and get one if you haven't yet.

In [7]:
# Paste your API key here. Remember to not share publicly
api_key = '9Y0OqZQHpfb7vVrypsRXChE8VZzVAmZMCtmbIrhu'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [10]:

news_examples = []


In [11]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
      extraction = co.generate(
          model='large',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
      return(extraction.generations[0].text[:-1])


coherenewsExtractor = cohereExtractor([e[1] for e in news_examples], 
                                       [e[0] for e in news_examples], [],
                                       "", 
                                       "extract the news title from the post:")

# Uncomment to inspect the full prompt:
# print(coherenewsExtractor.make_prompt('<input text here>'))

In [12]:
# This is what the prompt looks like:
print(coherenewsExtractor.make_prompt('<input text here>'))

<input text here>
extract the news title from the post:


## Getting the data


In [ ]:
num_posts = 10

movies_list = get_post_titles(size=num_posts, 
      after=str(int(datetime.datetime(2021,1,1,0,0).timestamp())), 
      before=str(int(datetime.datetime(2022,1,1,0,0).timestamp())), 
      subreddit="movies", 
      sort_type="score", 
      sort="desc")

# Show the list
movies_list

## Running the model
And now we loop over the posts and process each one of them with our extractor.

In [ ]:
results = []
for text in tqdm(movies_list):
    try:
        extracted_text = coherenewsExtractor.extract(text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

100%|██████████| 10/10 [00:10<00:00,  1.09s/it]


Let's look at the results:

In [ ]:
pd.DataFrame(data={'text': movies_list, 'extracted_text': results})

In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/movie_extraction_test_set_100.csv',index_col=0)
test_df

Let's run the extractor on these post titles (calling the API in parallel for quicker results):

In [ ]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['text']):
        extracted.append(str(i).strip())
# Save results
test_df['extracted_text'] = extracted

Let's look at some results:

In [ ]:
test_df.head()

Let's calculate the accuracy by comparing to the labeled examples

In [ ]:
# Compare the label to the extracted text
test_df['correct'] = (test_df['label'].str.lower() == test_df['extracted_text'].str.lower()).astype(int)

# Print the accuracy
print(f'Classification accuracy {test_df["correct"].mean() *100}%')

In [ ]:
test_df[test_df['correct']==0]

In [ ]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

print(classification_report(test_df['label'].str.lower(), test_df['extracted_text'].str.lower()))